# 浮動小数点数

## 浮動小数点数 (Floating Point Number)
計算機において小数は，**浮動小数点数**として実装されている．


In [10]:
0.0000123

1.23e-5

![fpn](fpn.svg)

浮動小数点数の標準規格はIEEE754で策定されており，次のような要素で構成される．

- 正規化数 (normal number)
- 非正規化数 (subnormal number)
- 特殊な値：`Inf` （無限大） や `NaN` (Not a Number) など．


## 単精度と倍精度

多くの計算機でハードウェアレベルで実装されている浮動小数点数は
 - 単精度 (single precision) 
 - 倍精度 (double precision) 

の２種類である．違いは表現可能な実数の範囲だけであり，倍精度の方が表現できる実数の範囲が広い．   
特別な理由がない限り，**倍精度**を用いる．

| 精度  | 仮数部     | 指数部 |10進桁数   | 型 |
|-------|----------|-------|------------|----|
| 単精度  | 23bit   | -1022 ～ +1023     | 約8桁    | `Float32` |
| 倍精度  | 52bit   | -126 ～ +127      | 約16桁   | `Float64` |


In [4]:
@show Float32(1)/3; # 単精度 Float32
@show 1/3   # 倍精度 Float64


Float32(1) / 3 = 0.33333334f0
1 / 3 = 0.3333333333333333


0.3333333333333333

## 正規化数，非正規化数
**正規化数**(normal number) とは，`1.0101e-5`（仮数部２進表記）のように先頭桁が1の浮動小数点数である．

- 最小の正の正規化数は 1e-1022
- 最大の正の正規化数は 1.11....1e1022 = $10^{1023} - 1$

正規化数を数直線上にプロットすると原点付近で相対的に大きな隙間が生まれる．  
この隙間を均等に埋めるのが**非正規化数** (subnormal number)である．  

非正規化数は `0.1101e-1022`のように先頭桁が0で，指数部が最小の浮動小数点数と定義される．


![subnormal](subnormal.svg)

## Overflow
倍精度で扱える最大の正の正規化数は表示しよう．

In [17]:
@show x_max = floatmax(Float64);

x_max = floatmax(Float64) = 1.7976931348623157e308


浮動小数点数の演算の結果が `x_max`を超えた場合，無限大を表す浮動小数点数 `Inf` に変換される．  
このような現象を overflow という．

In [15]:
2 * x_max

Inf

`Inf`の型は `Float64` である．

In [5]:
@show typeof(Inf)

typeof(Inf) = Float64


Float64

#### Note
`x_max`は $1.11\cdots 1_2 \times 2^{1023}$に等しい．
添字の2は2進数を表す．

`BigFloat`で計算して一致することを確認する．
仮数部を10進数に直すと，
$$
2^{0} + 2^{-1} + \cdots + 2^{-53} = \frac{1 - 2^{-54}}{1- 2^{-1}}
 = 2 - 2^{-54}.
$$


In [16]:
(2 - big(2.0)^(-54)) * big(2.0)^1023

1.797693134862315857833297452421029442208659153021342421048019300690888220491329e+308

確かに一致している．

#### Note
倍精度で扱える *最小の正の正規化数* を知りたい場合は，`floatmin`を使う．

In [17]:
x_min = floatmin(Float64)

2.2250738585072014e-308

## Underflow
最小の正の**非**正規化数は `eps(0.0)` で取得できる．

In [2]:
subnormal_min = eps(0.0)

5.0e-324

演算結果が この値を下回ると `0.0` に丸められることがあり，これを underflow という．

In [20]:
subnormal_min / 2

0.0

<div class="alert alert-block alert-warning">
Warning: `eps(0.0)`の出力する数値は誤りです．
内部的には正しい値が格納されているようなので表示上のバグだと思いますが，原因が分かった人は教えてください．
</div>

### NOTE
`subnormal_min` は $0.00\cdots 01_2 \times 2^{-1022}$に等しい．

仮数部を10進数で表すと，$2^{-52}$だから，$2^{-(1022 + 52)} = 2^{-1074}$.

`BigFloat`で表示してみる．

In [21]:
big(2.0)^(-1074)

4.940656458412465441765687928682213723650598026143247644255856825006755072702088e-324

In [1]:
big(eps(0.0))   # BigFloatで確認すると一致している．

4.940656458412465441765687928682213723650598026143247644255856825006755072702088e-324

In [5]:
bitstring(eps(0.0)) # bit列の表示．これは問題ない．

"0000000000000000000000000000000000000000000000000000000000000001"

## `NaN` (Not a Number)

浮動小数点数において未定義な演算等を行った結果，`NaN` という特殊な浮動小数点数が発生することがある．

例えば，0を0で割ると `Nan` が発生する．

In [23]:
0 / 0

NaN

ちなみに0以外の正の浮動小数点数をゼロで割ると`Inf`になる．

In [24]:
1 / 0

Inf

これ以外にも，`Inf`に関する四則演算の結果，`NaN` が発生することがある． 

In [25]:
@show Inf + Inf
@show Inf - Inf    # ∞ - ∞
@show Inf * Inf
@show Inf * 0      # ∞ ✕ 0
@show Inf / Inf    # ∞ ÷ ∞

Inf + Inf = Inf
Inf - Inf = NaN
Inf * Inf = Inf
Inf * 0 = NaN
Inf / Inf = NaN


NaN

不定形の極限に対応するような演算を行うと `NaN` が発生すると覚えておけばよい．

<div class="alert alert-block alert-warning">
Warning:
 `Inf` や `NaN` の発生を前提としたコード設計はしないようにしてください．

`Inf` や `NaN`の挙動を完全に把握することは非常に困難で，bug （プログラムにおける予期しない動作やエラー）
を生みやすいです．  
さらに，読み手にとっても理解しにくいコードになります．  
Juliaでは `isfinite(x)` で `x` が有限値であるかどうか， `isnan(x)`で `x` が `NaN` であるかどうかの判定ができます．
</div>

## 変数に`Inf`や`NaN`ではないことの確認

`Float64`型の変数は， `Inf`あるいは`NaN`といった小数ではない特殊値を取りうる．  
これらを判定するには，`isfinite()`，`isinf()`，`isnan()`を使う．

In [17]:
@show isfinite(1) isfinite(-Inf) isfinite(NaN);

isfinite(1) = true
isfinite(-Inf) = false
isfinite(NaN) = false


In [15]:
@show isnan(1) isnan(-Inf) isnan(NaN);

isnan(1) = false
isnan(-Inf) = false
isnan(NaN) = true


In [13]:
@show isinf(1) isinf(-Inf) isnan(NaN);

isinf(1) = false
isinf(-Inf) = true
isnan(NaN) = true


`ComplexF64`型も同様に判定できる．

In [21]:
@show isfinite(1) isfinite(1 - Inf*im)  isfinite(Inf + NaN*im)
@show isinf(1) isinf(1 - Inf*im)  isinf(Inf + NaN*im)
@show isnan(1) isnan(1 - Inf*im)  isnan(Inf + NaN*im);

isfinite(1) = true
isfinite(1 - Inf * im) = false
isfinite(Inf + NaN * im) = false
isinf(1) = false
isinf(1 - Inf * im) = true
isinf(Inf + NaN * im) = true
isnan(1) = false
isnan(1 - Inf * im) = false
isnan(Inf + NaN * im) = true
